In [1]:
from openai import OpenAI
from langchain_pinecone import PineconeEmbeddings, PineconeVectorStore

In [12]:
def get_food_nutrients(data):
    food_data = []

    for doc in data:
        food_info = {
            "Food Name": doc.page_content,
            "Nutrients": doc.metadata
        }
        food_data.append(food_info)

    return food_data

def generate_prompt(query, food_data):
    prompt_template = """
    Answer the user's query: {query}

    Using the food data information provided. Note that nutrient data is typically based on 100 grams of the food. If the user asks about serving sizes, distinguish between the standard 100-gram basis and any serving size mentioned in the data. Do not interpolate information.
    
    The food data may come from different sources, including branded foods, foundation foods (from chemical analysis), and survey foods (from dietary surveys). Only use the data relevant to the specific food they are asking about.
    
    Be aware that some foods may have inedible components (referred to as refuse) or may be processed in certain ways that impact their nutrient content or weight. Use any specific nutrient derivations or conversion factors provided when necessary, and cite the relevant details.
    
    For example, if the user asks about allergens, use the information from the ingredients or food components to answer accurately. You have access to data from FDA sources, so refer to specific nutrients, attributes, and details in your response. Don't use LaTex equations in your response.

    Make sure to start the response by mentioning the item you are referring to:

    Food Data: {food_data}
    """
    return prompt_template.format(query=query, food_data=food_data)

def get_completion(query, food_data):
    client = OpenAI()
    prompt = generate_prompt(query, food_data)

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return completion.choices[0].message.content

def retrieve_food_data_from_pinecone(query):
    embedding_model = PineconeEmbeddings(model="multilingual-e5-large")
    food_data_index = PineconeVectorStore.from_existing_index("food-data", embedding_model)
    retrieved_food_data = food_data_index.similarity_search(query, k=10)
    food_data = get_food_nutrients(retrieved_food_data)

    return food_data



In [14]:
user_query = "give me a breakdown of the nutrients in the cheerios and potential allergens "
retrieved_food_data = retrieve_food_data_from_pinecone(user_query)

response = get_completion(user_query, retrieved_food_data)
print(response)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000275723446D0>


Food Item: GMILLS HNY NUT CHEERIOS SWEETENED WHL GRN OAT CEREAL GNT SZ

Here is the nutrient breakdown per 100 grams:

- **Energy:** 393.0 kcal
- **Carbohydrate, by difference:** 78.57 g
- **Total Sugars:** 32.14 g
- **Total lipid (fat):** 5.36 g
  - Fatty acids, total monounsaturated: 1.79 g
  - Fatty acids, total polyunsaturated: 1.79 g
  - Fatty acids, total saturated: 0.0 g
  - Fatty acids, total trans: 0.0 g
- **Cholesterol:** 0.0 mg
- **Protein:** 7.14 g
- **Fiber, total dietary:** 7.1 g
  - Fiber, soluble: 4.0 g
- **Calcium, Ca:** 357.0 mg
- **Iron, Fe:** 16.07 mg
- **Magnesium, Mg:** 86.0 mg
- **Phosphorus, P:** 286.0 mg
- **Potassium, K:** 411.0 mg
- **Sodium, Na:** 571.0 mg
- **Zinc, Zn:** 13.39 mg
- **Vitamins:**
  - Vitamin A, IU: 1786.0 IU
  - Vitamin B-6: 1.786 mg
  - Vitamin B-12: 5.36 ug
  - Vitamin C, total ascorbic acid: 21.4 mg
  - Riboflavin: 1.518 mg
  - Thiamin: 1.0 mg
  - Vitamin D (D2 + D3), International Units: 143.0 IU

**Potential Allergens:**
Cheerios typica